In [58]:
import kfp
from google.cloud import aiplatform as vertex_ai
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental
from kfp.v2.google.client import AIPlatformClient

In [59]:
PROJECT_ID='ml-pipelines-showcase'

BUCKET_NAME='mlops-vertex-jsk'
PIPELINE_ROOT = "gs://{}/hearts/pipeline".format(BUCKET_NAME)
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [60]:
vertex_ai.init(
    project='ml-pipelines-showcase',
    location='us-central1',
    staging_bucket='gs://mlops-vertex-jsk',
)

In [61]:
num_neurons= "128"
dropout= 0.5



# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/{TIMESTAMP}"

MODEL_DISPLAY_NAME = f"hearts_train_deploy{TIMESTAMP}"


TRAINER_ARGS = ["--num_neurons", num_neurons, "--dropout", dropout, "--working_dir",WORKING_DIR]

print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)

['--num_neurons', '128', '--dropout', 0.5, '--working_dir', 'gs://mlops-vertex-jsk/hearts/pipeline/20211014115705'] gs://mlops-vertex-jsk/hearts/pipeline/20211014115705 hearts_train_deploy20211014115705


In [62]:
@component
def print_op(input1: str):
    print("training task: {}".format(input1))

In [63]:
@kfp.dsl.pipeline(name="hearts-train-endpoint-deploy" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-6:latest",
):

    train_task = print_op("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": TRAINER_ARGS,
                    "imageUri": "gcr.io/ml-pipelines-showcase/heart:training",
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-4",
                },
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )

In [64]:
from kfp.v2 import compiler  

In [65]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tabular regression_pipeline.json".replace(" ", "_"),
)

In [ ]:
DISPLAY_NAME = "hearts_" + TIMESTAMP

job = vertex_ai.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular regression_pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
)

job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/15357076341/locations/us-central1/pipelineJobs/hearts-train-endpoint-deploy20211014115705-20211014115707
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/15357076341/locations/us-central1/pipelineJobs/hearts-train-endpoint-deploy20211014115705-20211014115707')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/hearts-train-endpoint-deploy20211014115705-20211014115707?project=15357076341
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/15357076341/locations/us-central1/pipelineJobs/hearts-train-endpoint-deploy20211014115705-20211014115707 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.c